# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."
import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/bidding_V2-5158000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/binfo_V2-5158000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-999000
INFO:tensorflow:Re

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = '843.9765.A73.AK4'

# the auction goes:
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 1, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

auction = ["PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 2, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

auction = ["PAD_START","PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 3, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '1C', 'insta_score': 0.999}]
[{'call': '1C', 'insta_score': 0.999}]
[{'call': '1C', 'insta_score': 0.999}]
[{'call': '1C', 'insta_score': 0.999}]


in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
bid.samples

[]

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
8xx.9xxx.Axx.AKx AJT9xxx..KJ9.9xx .AKQx.QT8xx.Q8xx KQx.JT8xx.xx.JTx
8xx.9xxx.Axx.AKx AKQ9xx.T8.JT.Qxx T.KQJx.Q98xx.J9x Jxx.Axx.Kxx.T8xx
8xx.9xxx.Axx.AKx AK9xx.Qx.Qxx.9xx T.A8xx.K8x.QJT8x QJxx.KJT.JT9x.xx
8xx.9xxx.Axx.AKx KT9x.Kxx.JT9.J8x x.AQJx.Kxxx.Q9xx AQJxx.T8.Q8x.Txx
8xx.9xxx.Axx.AKx AJT9x.JTxx.9xx.Q x.AKQx.JTx.JT98x KQxx.8.KQ8x.xxxx
8xx.9xxx.Axx.AKx KTxx.QT8.Jxx.8xx x.Kxxx.KQxx.JTxx AQJ9x.AJ.T98.Q9x
8xx.9xxx.Axx.AKx AQxx.KJ8.Txx.9xx T.AQTx.K8xx.QJxx KJ9xx.xx.QJ9.T8x
8xx.9xxx.Axx.AKx AQJxx.8x.KJT.9xx .AKQx.Qxx.QJT8xx KT9xx.JTx.98xx.x
8xx.9xxx.Axx.AKx JTxx.Qxx.Kxx.Txx .AK8x.Q9xx.Q98xx AKQ9xx.JT.JT8.Jx
8xx.9xxx.Axx.AKx AKT9x.Jx.Txx.9xx x.AKxx.QJxx.JTxx QJxx.QT8.K98.Q8x
8xx.9xxx.Axx.AKx AKJ9xx.AT.T.JT8x x.KJ8x.K98x.Q9xx QTx.Qxx.QJxxx.xx
8xx.9xxx.Axx.AKx AQT9x.K8x.Jx.JTx .AQJT.KQ9xx.Q9xx KJxxx.xx.T8x.8xx
8xx.9xxx.Axx.AKx AQxx.J.QJ9x.Txxx .AQ8x.KTxxx.Q8xx KJT9xx.KTxx.8.J9
8xx.9xxx.Axx.AKx AJTxx.QT.K9x.Txx .AJxx.QJTx.J98xx KQ9xx.K8x.8xx.Qx
8xx.9xxx.Axx.AKx KT9xxx.Qxx.J